<a href="https://colab.research.google.com/github/Tanya-Verma/Apline_dashboard/blob/main/Copy_of_GPT_Dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch # we use PyTorch: https://pytorch.org

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

print(text[:1000])

chars = sorted(list(set(text)))

stoi = { ch:i for i,ch in enumerate(chars) }

--2026-02-02 07:23:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2026-02-02 07:23:07 (21.4 MB/s) - ‘input.txt.2’ saved [1115394/1115394]

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a

## Implement `get_batch` Function

### Subtask:
Implement the `get_batch` function, which will randomly sample batches of data from either the training or validation set. This function is crucial for efficiently feeding data to the model during training.

In [24]:
print("Generating 200 new tokens from the model:")
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(generate(m, context, max_new_tokens=200)[0].tolist()))

Generating 200 new tokens from the model:

Or my liveft ous
Ats trayinem thou, that yous me mordn.

LAnm; dithith alld se.

LAPThat ad dep west, his my me:
Py proodesss eentle cod the thaven, tuad hat hisBought day him thy that ould chald delf


## Define Toy Example Variables and Dummy Model

### Subtask:
Define `B`, `T`, `C`, `head_size`, and `xbow2` for toy examples. Also, provide a dummy PyTorch model (`m`) and optimizer to make the existing code snippets runnable.

In [12]:
B, T, C = 4, 8, 32 # batch, time, channels (embedding dimension)
head_size = 16
xbow2 = torch.zeros((B,T,C)) # Dummy xbow2 for testing purposes

# Dummy PyTorch model and optimizer to make snippets runnable
m = torch.nn.Linear(C, C) # A simple linear layer as a placeholder model
optimizer = torch.optim.Adam(m.parameters()) # A dummy optimizer

print(f"B (batch size): {B}")
print(f"T (sequence length): {T}")
print(f"C (channels/embedding dimension): {C}")
print(f"head_size: {head_size}")
print(f"Dummy model 'm' created: {m}")
print(f"Dummy optimizer created: {optimizer}")

B (batch size): 4
T (sequence length): 8
C (channels/embedding dimension): 32
head_size: 16
Dummy model 'm' created: Linear(in_features=32, out_features=32, bias=True)
Dummy optimizer created: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [11]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

# Test the get_batch function
xb, yb = get_batch('train')
print("Inputs (xb) shape:", xb.shape)
print("Targets (yb) shape:", yb.shape)
print("First batch inputs:", xb[0])
print("First batch targets:", yb[0])

Inputs (xb) shape: torch.Size([4, 8])
Targets (yb) shape: torch.Size([4, 8])
First batch inputs: tensor([45,  1, 57, 53,  1, 61, 47, 57])
First batch targets: tensor([ 1, 57, 53,  1, 61, 47, 57, 43])


# Task
Define the `encode` and `decode` functions, set `block_size` and `batch_size`, implement `get_batch`, define `B`, `T`, `C`, `head_size`, and `xbow2` for toy examples, and provide a dummy PyTorch model (`m`) and optimizer to make the existing code snippets runnable.

## Define Tokenization Functions

### Subtask:
Implement the `encode` and `decode` functions based on the `stoi` (string to integer) and `itos` (integer to string) mappings, which are crucial for processing the text data.


**Reasoning**:
The subtask requires defining `itos`, `encode`, and `decode` functions. I will start by creating the `itos` dictionary from the existing `stoi` dictionary and then implement the `encode` and `decode` functions as specified.



In [5]:
itos = { i:ch for i,ch in enumerate(chars) }

def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers

def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print("itos dictionary created successfully.")
print("encode function defined.")
print("decode function defined.")

data = torch.tensor(encode(text), dtype=torch.long)
print(f"Data tensor created with shape: {data.shape}")

itos dictionary created successfully.
encode function defined.
decode function defined.
Data tensor created with shape: torch.Size([1115394])


## Set Hyperparameters and Split Data

### Subtask:
Define `block_size` and `batch_size`. Split the `data` tensor into training and validation sets based on a 90/10 ratio.


**Reasoning**:
The subtask requires defining `block_size` and `batch_size`, and splitting the `data` tensor into training and validation sets. I will create a code block to accomplish these steps as per the instructions.



In [6]:
block_size = 8 # what is the maximum context length for predictions?
batch_size = 4 # how many independent sequences will we process in parallel?

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

print(f"block_size set to: {block_size}")
print(f"batch_size set to: {batch_size}")
print(f"Training data length: {len(train_data)}")
print(f"Validation data length: {len(val_data)}")

block_size set to: 8
batch_size set to: 4
Training data length: 1003854
Validation data length: 111540


# Task
Implement token and positional embedding layers to convert input tokens into vector representations suitable for a transformer model, using `nn.Embedding` for tokens and `nn.Parameter` for positional encodings.

## Implement Embedding Layers

### Subtask:
Create token and positional embedding layers to convert input tokens into rich vector representations suitable for the transformer model.


**Reasoning**:
The subtask requires implementing token and positional embedding layers. I will define the vocabulary size, and then instantiate the `nn.Embedding` for token embeddings and `nn.Parameter` for positional embeddings as per the instructions.



In [13]:
import torch
import torch.nn as nn

vocab_size = len(chars)

token_embedding_table = nn.Embedding(vocab_size, C)
position_embedding_table = nn.Parameter(torch.zeros(block_size, C))

print(f"Vocab size: {vocab_size}")
print(f"Token embedding table created with shape: {token_embedding_table.weight.shape}")
print(f"Positional embedding table created with shape: {position_embedding_table.shape}")

Vocab size: 65
Token embedding table created with shape: torch.Size([65, 32])
Positional embedding table created with shape: torch.Size([8, 32])


## Implement Attention Head

### Subtask:
Implement a single self-attention head, including the creation of query, key, and value linear layers, and the masked attention mechanism to prevent information leakage from future tokens.


**Reasoning**:
The subtask requires implementing the `Head` class for a single self-attention head, including its `__init__` and `forward` methods, as specified in the instructions. This code block will define the class with linear layers, attention calculation, masking, softmax, and dropout.



In [14]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(C, head_size, bias=False)
        self.query = nn.Linear(C, head_size, bias=False)
        self.value = nn.Linear(C, head_size, bias=False)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape # Unpack dimensions from input x
        k = self.key(x)   # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)
        v = self.value(x) # (B, T, head_size)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

        tril = torch.tril(torch.ones(T, T))
        wei = wei.masked_fill(tril == 0, float('-inf')) # (B, T, T)
        wei = torch.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        out = wei @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
        return out

print("Head class defined successfully.")


Head class defined successfully.


## Implement Multi-Head Attention

### Subtask:
Combine multiple attention heads into a Multi-Head Attention module. This involves splitting and concatenating outputs from individual heads and a final linear projection.


**Reasoning**:
The subtask requires implementing the `MultiHeadAttention` class by combining multiple `Head` instances, adding a projection layer, and a dropout layer, as specified in the instructions. This code block will define the class and its methods.



In [15]:
class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, C)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

print("MultiHeadAttention class defined successfully.")

MultiHeadAttention class defined successfully.


## Implement Feed-Forward Network

### Subtask:
Create a simple feed-forward network, which is typically a two-layer neural network with a ReLU activation, to be used within each transformer block.


**Reasoning**:
The subtask requires implementing a `FeedFoward` class, which will consist of two linear layers with a ReLU activation and a dropout layer, as detailed in the instructions.



In [16]:
import torch.nn.functional as F

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, C):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(C, 4 * C),
            nn.ReLU(),
            nn.Linear(4 * C, C),
            nn.Dropout(0.1),
        )

    def forward(self, x):
        return self.net(x)

print("FeedFoward class defined successfully.")

FeedFoward class defined successfully.


## Implement Transformer Block

### Subtask:
Assemble the Multi-Head Attention and Feed-Forward Network components, along with layer normalization and residual connections, into a complete Transformer Block.


**Reasoning**:
The subtask requires assembling the `MultiHeadAttention` and `FeedFoward` components into a `Block` class, including layer normalization and residual connections. I will define the `Block` class with its `__init__` and `forward` methods as per the instructions, calculating `num_heads` based on `C` and `head_size`.



In [17]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, C, head_size):
        super().__init__()
        num_heads = C // head_size
        self.sa = MultiHeadAttention(num_heads, head_size)
        self.ffwd = FeedFoward(C)
        self.ln1 = nn.LayerNorm(C)
        self.ln2 = nn.LayerNorm(C)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # Apply LayerNorm before attention, then add residual
        x = x + self.ffwd(self.ln2(x)) # Apply LayerNorm before feed-forward, then add residual
        return x

print("Block class defined successfully.")

Block class defined successfully.


## Implement GPT Model

### Subtask:
Build the full GPT model by stacking multiple Transformer Blocks, adding the initial embedding layers, and a final linear layer for language modeling head to predict the next token.


**Reasoning**:
The subtask requires implementing the full GPT model. I will define the `GPT` class, initializing the embedding layers, stacking multiple `Block` instances, adding a final layer normalization, and a linear head for token prediction. The `forward` method will handle combining embeddings, passing data through blocks, and computing logits and loss as per the instructions.



In [18]:
class GPT(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, C)
        self.position_embedding_table = nn.Embedding(block_size, C)
        num_heads = C // head_size
        self.blocks = nn.Sequential(*[Block(C, head_size) for _ in range(3)]) # Stack 3 Transformer Blocks
        self.ln_f = nn.LayerNorm(C) # final layer norm
        self.lm_head = nn.Linear(C, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        loss = None
        if targets is not None:
            # Reshape logits and targets for F.cross_entropy
            logits = logits.view(B*T, vocab_size)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

print("GPT class defined successfully.")

GPT class defined successfully.


## Training Loop

### Subtask:
Set up the training loop, including defining the loss function (CrossEntropyLoss), an optimizer (AdamW), and iterating over batches of data to train the model.


**Reasoning**:
The instructions require instantiating the GPT model, moving it to the correct device (CUDA if available, else CPU), and defining the AdamW optimizer with a specified learning rate. This code block will perform all these initial setup steps for the training loop.



In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
m = GPT()
m.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

print(f"Model instantiated and moved to: {device}")
print(f"Optimizer created: {optimizer}")

Model instantiated and moved to: cpu
Optimizer created: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: True
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)


**Reasoning**:
The subtask requires iterating over batches of data to train the model. This involves getting a batch, performing a forward pass, calculating loss, performing a backward pass, and updating the model parameters within a training loop.



In [20]:
epochs = 5000 # Number of training iterations

for iter in range(epochs):
    # sample a batch of data
    xb, yb = get_batch('train')

    # move data to the same device as the model
    xb, yb = xb.to(device), yb.to(device)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if iter % 500 == 0:
        print(f"Iteration {iter}, Loss: {loss.item():.4f}")

print(f"Training complete. Final Loss: {loss.item():.4f}")

Iteration 0, Loss: 4.3853
Iteration 500, Loss: 3.0012
Iteration 1000, Loss: 2.6455
Iteration 1500, Loss: 2.8824
Iteration 2000, Loss: 2.7084
Iteration 2500, Loss: 2.5716
Iteration 3000, Loss: 2.0871
Iteration 3500, Loss: 2.1791
Iteration 4000, Loss: 2.1737
Iteration 4500, Loss: 2.3662
Training complete. Final Loss: 2.6273


## Model Generation Function

### Subtask:
Implement a function to generate text from the trained GPT model, allowing the model to sample tokens autoregressively.


**Reasoning**:
The subtask requires implementing a `generate` function for the GPT model. This function will take an initial sequence (`idx`), and autoregressively generate `max_new_tokens` by repeatedly predicting the next token, sampling from the predictions, and appending it to the sequence. Finally, I will test the function with an example to demonstrate its functionality.



In [21]:
import torch.nn.functional as F

def generate(model, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get the predictions
        logits, loss = model(idx_cond) # `model` refers to `m` in this context
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

# Test the generation function
# Create a starting prompt (e.g., a single newline character to start fresh)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(generate(m, context, max_new_tokens=100)[0].tolist()))


And.
Carl get whe lise ordovers hes the thair word yeald tatlem lild to well that
Tou bot and ocoye.


## Train and Evaluate Model

### Subtask:
Execute the training loop for a specified number of iterations and periodically evaluate the model's performance on the validation set. After training, use the model to generate a sample text.


### Define and Integrate Evaluation Function

**Reasoning**:
The subtask requires defining an evaluation function `estimate_loss()` to assess the model's performance on both training and validation sets. This function will be integrated into the existing training loop to provide periodic updates on the model's generalization capabilities.

### Generate Text After Training

**Reasoning**:
After the training is complete, the subtask requires using the trained model to generate a sample text. This will demonstrate the model's learned ability to produce new sequences based on a given context, by calling the previously defined `generate` function.

**Reasoning**:
The subtask requires defining the `estimate_loss` function. This code block will implement the function as specified, which includes setting the model to evaluation mode, calculating losses on training and validation sets over several batches, and then returning the model to training mode.



In [22]:
eval_iters = 200 # Number of batches to average loss over for evaluation

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval() # Set model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X, Y = X.to(device), Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # Set model back to training mode
    return out

print("estimate_loss function defined successfully.")

estimate_loss function defined successfully.


**Reasoning**:
The subtask requires integrating the `estimate_loss` function into the training loop for periodic evaluation and then generating text after training. This code block will modify the existing training loop to include periodic calls to `estimate_loss` and will add the text generation step at the end.



In [23]:
epochs = 5000 # Number of training iterations

for iter in range(epochs):
    # evaluate loss on train and val sets periodically
    if iter % 500 == 0 or iter == epochs - 1:
        losses = estimate_loss(m)
        print(f"Iteration {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # move data to the same device as the model
    xb, yb = xb.to(device), yb.to(device)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True);
    loss.backward()
    optimizer.step()

print(f"\nTraining complete after {epochs} iterations.")

# Generate text after training
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("\nGenerated text:")
print(decode(generate(m, context, max_new_tokens=100)[0].tolist()))

Iteration 0: train loss 2.2708, val loss 2.2899
Iteration 500: train loss 2.2387, val loss 2.2567
Iteration 1000: train loss 2.2553, val loss 2.2441
Iteration 1500: train loss 2.2429, val loss 2.2560
Iteration 2000: train loss 2.2325, val loss 2.2492
Iteration 2500: train loss 2.2018, val loss 2.2340
Iteration 3000: train loss 2.1850, val loss 2.2186
Iteration 3500: train loss 2.1895, val loss 2.2133
Iteration 4000: train loss 2.1788, val loss 2.1955
Iteration 4500: train loss 2.1893, val loss 2.1923
Iteration 4999: train loss 2.1344, val loss 2.1962

Training complete after 5000 iterations.

Generated text:

Will.

Switk I kust ould mang mousbedes ot if boster:
When coutenst Fighen, your frieasu and thred s


## Summary:

### Data Analysis Key Findings

*   **Embedding Layers**: The model successfully initialized token embeddings (`token_embedding_table`) with a shape of `[65, 32]` (vocab\_size, embedding\_dimension) and positional embeddings (`position_embedding_table`) with a shape of `[8, 32]` (block\_size, embedding\_dimension). The vocabulary size was determined to be 65.
*   **Transformer Components**:
    *   A single self-attention `Head` was implemented with query, key, and value linear layers and a masked attention mechanism.
    *   The `MultiHeadAttention` module was successfully created, combining multiple `Head` instances and applying a final linear projection.
    *   A `FeedFoward` network was implemented as a two-layer neural network with ReLU activation and dropout.
    *   These components were integrated into a `Block` module, including layer normalization and residual connections, to form a complete Transformer block.
*   **GPT Model Construction**: The full `GPT` model was assembled by stacking 3 Transformer `Block` instances, along with the initial token and positional embedding layers, a final layer normalization, and a linear language modeling head for next-token prediction.
*   **Training Setup**:
    *   The `GPT` model was instantiated and moved to the 'cpu' device.
    *   An `AdamW` optimizer was configured with a learning rate of $1e-3$.
*   **Training Progress (Initial Loop)**: After 5000 training iterations, the loss decreased from an initial value of $4.3853$ to a final value of $2.6273$, indicating that the model was learning.
*   **Evaluation Integration**: An `estimate_loss` function was successfully defined and integrated into the training loop to periodically evaluate the model's performance on both training and validation sets. At iteration 0, the training loss was $2.2708$ and validation loss was $2.2899$. At iteration 1000, these had decreased to $2.2553$ (train) and $2.2441$ (val), demonstrating continued learning.
*   **Text Generation**: A `generate` function was implemented, allowing the trained `GPT` model to produce autoregressive text sequences of a specified length (e.g., 100 new tokens) based on an initial context.

### Insights or Next Steps

*   The foundational architecture of a GPT-like transformer model has been successfully implemented and partially trained, demonstrating the core components are functional.
*   To further improve model performance and generated text quality, comprehensive hyperparameter tuning (e.g., learning rate, number of blocks, head size, dropout rates) and training on a larger, more diverse dataset are essential next steps.
